In [1]:
from data_interfaces.unpaired_interfaces.face_to_simpson_interface import FaceToSimpsonDataset
from torch.utils.data import DataLoader
import torch
from torchvision.utils import save_image
from utils import bcolors
from models.CycleGAN import CycleGAN
import os
import shutil

In [2]:
!git clone https://github.com/yhlleo/GAN-Metrics/

Cloning into 'GAN-Metrics'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 98 (delta 43), reused 72 (delta 20), pack-reused 0
Receiving objects: 100% (98/98), 68.81 KiB | 1.17 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [3]:
# Where to save images generated by the generator (G: X -> Y)
generated_images_dir = "generated"

# A file containing the paths of all the images generated
generated_filenames_file_path = "gtlist.txt"

# A file containing the paths of all test set's images
test_filenames_filepath = "predlist.txt"

os.makedirs(generated_images_dir, exist_ok=True)

generated_filenames_file = open(generated_filenames_file_path, "w")

# Set Torch Device
device = "cpu"
execution_device = torch.device(device)

# Define and create Test Dataset
test_data = FaceToSimpsonDataset("datasets/my_simpson_dataset/testB", "datasets/my_simpson_dataset/testA", apply_transforms=False)

# Create Data Loader
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=True)

# Create Model
cycleGAN = CycleGAN(training_phase=False, device=execution_device, save_dir="checkpoints", generated_images_dir=generated_images_dir,
                    generated_filenames_file=generated_filenames_file, lambda_cycle_loss=10.0, init_gain=0.02)

print(f"Device used: {execution_device}")

print("Testing...")

cycleGAN.load_checkpoints(best_model=True, restore_training=False)

for i, input_data in enumerate(test_dataloader):
    x_images, y_images = input_data
    cycleGAN.set_input_tensors(x_images, y_images)
    cycleGAN.test_step()


generated_filenames_file.close()

Device used: cpu
Testing...


In [12]:
!find "$(realpath datasets/my_simpson_dataset/testA)" -name '*.png' > gtlist.txt
!find "$(realpath generated)" -name '*.png' > predlist.txt

In [16]:
!echo "Le immagini dei dataset devono essere 100 per entrambi"
!echo "Files size gtlist $(cat gtlist.txt | wc -l | tr -d '\n'); predlist $(cat predlist.txt | wc -l | tr -d '\n');"

!python GAN-Metrics/eval.py --metric fid --pred_list predlist.txt --gt_list gtlist.txt --resize 299 --gpu_id ''

Le immagini dei dataset devono essere 100 per entrambi
Files size gtlist      100; predlist      100;
/Users/antoninoagostino/miniconda3/envs/computervision/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/antoninoagostino/miniconda3/envs/computervision/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Propagating batch 1/13
Propagating batch 2/13
Propagating batch 3/13
Propagating batch 4/13
Propagating batch 5/13
Propagating batch 6/13
Propagating batch 7/13
Propagating batch 8/13
Propaga